# Pipeline for a data warehouse
### Data Engineering Capstone Project

#### Project Summary
This is the capstone project of the udacity nanodegree for data engineering. 
The aim of the project is to apply learned skills during the course. This project will show how to load and transform data from four different data sources, load the data in spark apply quality checks and store the data into a star schema so that it can be used for BI apps and ad hoc analysis.

The project follows the follow steps:
* _Step 1_: Scope the Project and Gather Data
* _Step 2_: Explore and Assess the Data
* _Step 3_: Define the Data Model
* _Step 4_: Run ETL to Model the Data
* _Step 5_: Complete Project Write Up

### Step 1: Scope the Project and Gather Data

#### Scope 
As one of the acceptance criteria for this project is to handle at least 1 million rows and two different data sources and file formats, we will use the data sources are provided by Udacity. 
The scope of this project is to create a star schema source-of-truth database so that it can be used for BI apps and ad hoc analysis.

#### Describe and Gather Data 

The main dataset includes data about 
- _Immigration in the United States of America_ from [here](https://www.trade.gov/national-travel-and-tourism-office).

Supplementary datasets provided are:

- U.S. city _demographics_ from [here](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)

- _Temperature_ data from [here](https://www.trade.gov/national-travel-and-tourism-office)

- Data on _airport codes_ from [here](https://datahub.io/core/airport-codes#data)

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, desc, asc, to_date
from pyspark.sql.types import StructType, StructField as Fld, StringType as Str, IntegerType as Int, ShortType as Short, DoubleType as Dbl, ByteType as Bt
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd


### Step 2 - Explore and Asses the Data
The exploration of the used datasets including descriptions of the above mentioned data can be found in `data/exploration.ipynb`.

#### Important findings of the exploration

##### Immigrant data:
 -- dates are provided in SAS timestamps and need to be transformed

 -- there are many columns that have a lot of null values or are marked with CIC does not use", hence can be skipped. As the immigrant data is provided in parquet files, we can only read in the columns we need for this project.

 -- We might need helper tables to provide data for the

 --- visa codes: 1 = Business 2 = Pleasure 3 = Student 
 
 --- travel code:  1 = 'Air' 2 = 'Sea' 3 = 'Land' 9 = 'Not 

##### City demographics data:
- the "count" column can be skipped, as it was used for some other analysis
- the "number of veterans" can be also skipped as the main focus lies on the immigrant data.
 
##### Weather data:

- The temperature data is provided on a monthly basis
- The "dt" gives us the the date as a string. We need to transform it to datetime. As it holds data from 1860 on, we only want to keep the rows from 1960 on. 
- We only want to use the rows where the country is US.
- The name of the city is used to map to the weather and then store to weather table in order to get a star schema.

##### Possible next steps:
Connect airport data based on lat and long. Maybe with some margin.
We could also create a helper table to map the ports to cities.

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

The data shall be represented in a star schema. The advantage is, that it is easy to query an easy to understand.

// todo add image of star schema


#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

In order to pipe the data into the chosen data model, we will use Spark. This has the advantage that we can do the transformations in memory, before writing the data to the tables.
1. Load only the needed columns from the immigrant files into a spark dataframe. 
2. Create the fact_immigration table and write to parquet files.
3. Transform the given SAS timestamps and create a dim_date table. And write to parquet file.
4. Load the data for the other dimension tables. Write them to parquet files. 

Whenever possible use a schema when loading, to have a validation of the incoming data.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.


In [ ]:
spark = SparkSession \
    .builder \
    .appName("Capstone Project") \
    .getOrCreate()

In [ ]:
sas_data_part = '/Users/joebsbar/Documents/GitBarbara/data-engineering-nd/capstone-project/data/sas_data/part-00000-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet'

# instead of all 28 columns, we only need 10 for the tables we are going to create
city_schema = StructType([
    Fld("cicid", Short(), False),
    Fld("arrdate", Str()),
    Fld("depdate", Str()),
    Fld("i94cit", Short(), False),
    Fld("i94res", Short()),
    Fld("i94port", Str()),
    Fld("fltno", Str()),
    Fld("biryear", Short()),
    Fld("gender", Bt()),
    Fld("i94visa", Bt())
])

# read only the needed data for our fact table
df_immigration = spark.read.parquet(sas_data_part).select('cicid', 'arrdate', 'depdate', 'i94cit', 'i94res', 'i94port', 'fltno', 'biryear', 'gender', 'i94visa')
#df_immigration.count()
#df_immigration.printSchema()
#df_immigration.head()
print(('rows', df_immigration.count(), 'columns', len(df_immigration.columns)))


In [ ]:
from pyspark.sql.types import DateType

# df_immigration.createOrReplaceTempView("immigration")

# add timestamp arrival_date
#transform and add new column
# https://online.stat.psu.edu/stat481/book/export/html/702
# https://stackoverflow.com/questions/36412864/convert-numeric-sas-date-to-datetime-in-pandas
# only immigrations if us

print(df_immigration)
print(('rows', df_immigration.count(), 'columns', len(df_immigration.columns)))

@udf
def sas_to_timestamp(date_sas: float) -> int:
    """
    params: data_sas: a date in sas format
    return: a timestamp in seconds
    """

    if date_sas:
        datetime = pd.to_timedelta((date_sas), unit='D') + pd.Timestamp('1960-1-1')
        timestamp = datetime.timestamp()
        return timestamp
        
df_immigration = df_immigration.withColumn('arrival_ts', sas_to_timestamp(df_immigration['arrdate']))
df_immigration = df_immigration.withColumn('departure_ts', sas_to_timestamp(df_immigration['depdate']))
df_immigration.show(1)
print(('rows', df_immigration.count(), 'columns', len(df_immigration.columns)))


In [ ]:

# create fact_immigration and write to parquet
fact_immigration = df_immigration.select('cicid', 'arrival_ts', 'departure_ts', 'i94cit', 'i94res', 'i94port', 'fltno').dropDuplicates()

fact_immigration.head()
fact_immigration.write.parquet('./fact_immigration', mode='overwrite')

In [ ]:
from pyspark.sql import functions as F

# get all existing timestamps and drop duplicats
df_time = df_immigration.select('arrival_ts', 'departure_ts')
df_time = df_time.select('arrival_ts').unionAll(df_time.select('departure_ts'))
df_time = df_time.withColumnRenamed('arrival_ts', 'ts')
df_time.dropDuplicates()

# create dim_time table
dim_time = df_time.select('ts') \
            .withColumn('date', F.from_unixtime(F.col('ts')/1000)) \
            .withColumn('year', F.year('ts')) \
            .withColumn('month', F.month('ts')) \
            .withColumn('week', F.weekofyear('ts')) \
            .withColumn('weekday', F.dayofweek('ts')) \
            .withColumn('day', F.dayofyear('ts')) \
            .withColumn('hour', F.hour('ts'))

print(('rows', dim_time.count(), 'columns', len(dim_time.columns)))

In [ ]:
dim_time.write.parquet('./dim_time', mode='overwrite', partitionBy=['year', 'month'])

## Temperature Data

In [ ]:
import datetime

weather_schema = StructType([
    Fld("dt", Str()),
    Fld("AverageTemperature", Str()),
    Fld("AverageTemperatureUncertainty", Dbl()),
    Fld("City", Int()),
    Fld("Country", Int()),
    Fld("Latitude", Int()),
    Fld("Longitude", Int())
])

fname = 'data/GlobalLandTemperaturesByCity.csv'
df_temperature = spark.read.option("delimiter", ";").csv(fname, schema=weather_schema, header=True)
df_temperature_us = df_temperature[df_temperature["Country"] == "United States"]

year_format = '%Y-%m-%d'
# transform string type to datetype

df_temperature_us = df_temperature_us.withColumn("date", to_date("dt"))

#df_temperature_us = df_temperature_us.filter(df_temperature_us.date >= "1970-01-01")

# df_temperature_us.withColumn('date', get_date)
# get rid of all rows earlier than 1960
# df_temperature_us_data = df_temperature_us_data[df_temperature_us_data["date"] >= '1960-01-01' ]

print(df_temperature_us.schema)
print(('rows', df_temperature_us.count(), 'columns', len(df_temperature_us.columns)))


#display(df_temperature_us)
# todo transform datetime to have year, month and dt
# todo get rid of country
# starts with 1820 -> get rid of all dates previous than 
# toto get rid of all dates before 1960 - 1820-01-01 
#create dim_weather table
# store dim_weather table

In [ ]:
import datetime



## US City Demographics

In [ ]:
fname = 'data/us-cities-demographics.csv'


In [ ]:
city_schema = StructType([
    Fld("city_name", Str()),
    Fld("state", Str()),
    Fld("median_age", Dbl()),
    Fld("male_population", Int()),
    Fld("total_population", Int()),
    Fld("foreign_born", Int()),
    Fld("average_householdsize", Int()),
    Fld("state_code", Int()),
    Fld("race", Str()),
])

dim_city =  spark.read.option("delimiter", ";").csv(fname, schema=city_schema, header=True)
display(df_city)

# or read with header and map
# df_city =  spark.read.option("header",True).option("delimiter", ";").csv(fname)
# dim_city = df_city.withColumnRenamed("City","city_name") \
#             .withColumnRenamed("State","state") \
#             .withColumnRenamed("Median Age", "median_age") \
#             .withColumnRenamed("Male Population", "male_population") \
#             .withColumnRenamed("Female Population", "female_population") \
#             .withColumnRenamed("Total Population", "total_population") \
#             .withColumnRenamed("Foreign-born", "foreign_born") \
#             .withColumnRenamed("Average Household Size", "average_householdsize") \
#             .withColumnRenamed("State Code", "state_code") \
#             .withColumnRenamed("Race", "race") 

city_code_df = df_immigration["i94cit"]

# TODO WHY NOT JOINABLE???
dim_city_with_code = dim_city.join(city_code_df)

print(dim_city_with_code)

# DIM IMMIGRANT PERSON

In [ ]:
# create fact_immigration and write to parquet
dim_immigrant_person = df_immigration.select('cicid', 'biryear', 'gender', 'i94visa').dropDuplicates()

dim_immigrant_person.head()
dim_immigrant_person.writeparquet('./dim_immigrant_person', mode='overwrite')

### Step 2: Explore and Assess the Data
#### Explore the Data 
The data exploration can be found in `data/exploration.ipynb`

#### Cleaning Steps

#### Cleaning Steps - Immigration Data





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

The code for the pipelines can be found in `etl.py`

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.